In [ ]:
import cv2
import numpy as np
import os

prototxt_path = "deploy.prototxt"
caffemodel_path = "res10_300x300_ssd_iter_140000_fp16.caffemodel"
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

def cvDnnDetectFaces(image, opencv_dnn_model, min_confidence=0.5, display=True):
    image_height, image_width, _ = image.shape
    output_image = image.copy()

    preprocessed_image = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(300, 300),
                                               mean=(104.0, 117.0, 123.0), swapRB=False, crop=False)

    opencv_dnn_model.setInput(preprocessed_image)
    results = opencv_dnn_model.forward()

    detected_faces = []

    for face in results[0, 0]:
        face_confidence = face[2]

        if face_confidence > min_confidence:
            bbox = face[3:]

            x1 = int(bbox[0] * image_width)
            y1 = int(bbox[1] * image_height)
            x2 = int(bbox[2] * image_width)
            y2 = int(bbox[3] * image_height)

            # Draw the rectangle on the output image
            cv2.rectangle(output_image, pt1=(x1, y1), pt2=(x2, y2), color=(0, 255, 0), thickness=image_width // 200)

            # Append face coordinates to the list
            detected_faces.append((x1, y1, x2, y2))

    if display:
        cv2.imshow("Output", output_image)
    else:
        return output_image, detected_faces

DATA_DIR = 'drowsiness_detector/data/train1'
OUTPUT_DIR = 'drowsiness_detector/cropped_data1'
os.makedirs(OUTPUT_DIR, exist_ok=True)

for class_name in os.listdir(DATA_DIR):
    class_dir = os.path.join(DATA_DIR, class_name)
    output_class_dir = os.path.join(OUTPUT_DIR, class_name)
    os.makedirs(output_class_dir, exist_ok=True)

    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        img = cv2.imread(img_path)

        if img is not None:
            output_image, detected_faces = cvDnnDetectFaces(img, net, display=False)

            for i, face_info in enumerate(detected_faces):
                x1, y1, x2, y2 = face_info
                output_path = os.path.join(output_class_dir, f"face_{i}_{img_name}")

                if y2 > 0 and x2 > 0 and y2 <= img.shape[0] and x2 <= img.shape[1]:
                    cv2.imwrite(output_path, img[y1:y2, x1:x2])
                else:
                    print(f"Skipping saving image {output_path} due to empty or invalid region")

cv2.destroyAllWindows()